In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import style
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv("/kaggle/input/country-regional-and-world-gdp/gdp_csv.csv")

df.head(10)

**Finding out the names of the countries in the dataset**

In [ ]:
df["Country Name"].unique()

Since the dataset is a mixture of regions and countries, it would make sense to seperate these into two sepeate datasets

**Separating the dataset into "regions" and "countries" sub-datasets**

In [ ]:
region = ['Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa',
       'Middle East & North Africa (excluding high income)',
       'Middle East & North Africa (IDA & IBRD countries)',
       'Middle income', 'North America', 'OECD members',
       'Other small states', 'Pacific island small states',
       'Post-demographic dividend', 'Pre-demographic dividend',
       'Small states', 'South Asia', 'South Asia (IDA & IBRD)',
       'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
       'Sub-Saharan Africa (IDA & IBRD countries)', 'Upper middle income',
       'World']

In [ ]:
regions = df[df['Country Name'].isin(region)]
regions.reset_index(inplace=True, drop=True)

countries = df[~df['Country Name'].isin(region)]
countries.reset_index(inplace=True, drop=True)

In [ ]:
regions.head()

In [ ]:
countries.head()

The focus of this notebook will be on the analysis of individual countries as opposed to regions, therefore only the dataset with individual countries will be cleaned and analysed from this point forward.

**Finding out if each country has the same number of yearly data**

In [ ]:
countries.groupby("Country Name")["Year"].count()

Each country and region does not have the same number of yearly data since some countries have data going back 57 years while other countries have data going back only less than 27 years, subsequent analysis would not yield accurate results due to this mismatch. Therefore, it would make sense to create a sub-dataset with countries that have the same number of yearly data.

**Creating a sub-dataset which only includes countries who have the latest GDP values from 2016**

In [ ]:
countries2 = countries.groupby("Country Name", as_index=False)["Year"].max()

countries3 = countries2[countries2["Year"]==2016]

countries4 = countries3["Country Name"]

countries5 = countries[countries["Country Name"].isin(countries4)]

countries5

**Finding out which countries in the sub-dataset has the lowest number of years' worth of data and subsequently removing them so that we remain with an equal and sufficient amount of data for each country**

In [ ]:
min_year = countries5.groupby("Country Name", as_index=False)["Year"].min()

min_year["Year"].max()

2013 would not be a good starting year for our analysis since it would only give us 4 years worth of data for each country, therefore we would need to remove this country and repeat this step till we are left with a starting year that would leave us with sufficient amount of data for each country.

In [ ]:
min_year[min_year["Year"]==2013]

In [ ]:
countries6 = countries5[countries5["Country Name"]!="Somalia"]

min_year = countries6.groupby("Country Name", as_index=False)["Year"].min()

min_year["Year"].max()

2007 is still not a good starting year, therefore we will repeat the process

In [ ]:
min_year[min_year["Year"]==2007]

In [ ]:
countries7 = countries6[countries5["Country Name"]!="Nauru"]

min_year = countries7.groupby("Country Name", as_index=False)["Year"].min()

min_year["Year"].max()

In [ ]:
min_year[min_year["Year"]==2002]

In [ ]:
countries8 = countries7[~countries7["Country Name"].isin(["American Samoa","Guam","Northern Mariana Islands"])]

min_year = countries8.groupby("Country Name", as_index=False)["Year"].min()

min_year["Year"].max()

In [ ]:
min_year[min_year["Year"]==2001]

In [ ]:
countries9 = countries8[countries8["Country Name"]!="Sao Tome and Principe"]

min_year = countries9.groupby("Country Name", as_index=False)["Year"].min()

min_year["Year"].max()

I believe 2000 is a great starting year for our analysis since it gives us 17 years worth of data for each country

In [ ]:
country = countries9[countries9["Year"]>=2000]

country

**Making sure that the new dataset does not have any missing years for each of the countries**

In [ ]:
missing = country.groupby("Country Name", as_index=False)["Year"].count()

missing[missing["Year"]!=17]

Since the three countries above are missing some years as part of their data, they would need to be removed

**Removing Afghanistan, Democratic Republic of Congo and Iraq from the dataset and creating our final dataset that will be used for the analysis**

In [ ]:
country_new = pd.DataFrame(country[~country["Country Name"].isin(["Afghanistan","Congo, Dem. Rep.","Iraq"])])

country_new.reset_index(drop=True, inplace=True)

country_new["Country Name"].nunique()

The new dataset has 181 countries and their corresponding 17 years worth of data from the year 2000 to 2016

In [ ]:
country_new.head()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

**Finding out the top 15 countries with the highest GDP in 2016**

In [ ]:
countries_2016 = country_new[country_new["Year"]==2016]

countries_2016.sort_values("Value", axis=0, ascending=False, inplace=True)

countries_2016.reset_index(drop=True, inplace=True)

countries_2016_highest = countries_2016.head(15)

countries_2016_highest

fig = px.bar(countries_2016_highest, x="Country Name", y="Value", color="Country Name",
             color_discrete_sequence=px.colors.qualitative.Vivid)

fig.update_layout(title_text="Top 15 Countries with the highest GDP (2016)", title_font_size=22,
                  height=800, width=980, yaxis_title="GDP($)", xaxis_title="Country",
                  title_y=0.97, title_x=0.45)

fig.show()

countries_2016_highest

**Creating a Heatmap to represent countries GDPs in 2016**

In [ ]:
fig = px.choropleth(countries_2016, locations="Country Name", locationmode='country names', color="Value",
                    color_continuous_scale=px.colors.sequential.Redor)

fig.update_layout(title_text="Heatmap of GDPs for Countries in 2016", title_font_size=24,
                  height=800, width=1000, yaxis_title="GDP($)", xaxis_title="Country",
                  title_y=0.85, title_x=0.45)

fig.show()

The reason there are some countries that are unshaded is because we had removed some of them earlier to create a dataset with countries that have the same number of years worth of data

**Finding out the trend of GDP from 2000 to 2016 for the top 15 countries with the highest GDP in 2016**

In [ ]:
countries_highest_trend = country_new[country_new["Country Name"].isin(["United States","China","Japan","Germany","United Kingdom",
                                                                       "France","India","Italy","Brazil","Canada","Korea, Rep.",
                                                                       "Russian Federation","Spain","Australia","Mexico"])]

fig = px.bar(countries_highest_trend, x="Country Name", y="Value", color="Country Name",
             color_discrete_sequence=px.colors.qualitative.Dark24,
             animation_frame="Year", animation_group="Country Name")

fig.update_layout(title_text="Trends of Top 15 Countries with the highest GDP in 2016", title_font_size=22,
                  height=700, width=980, yaxis_title="GDP($)", xaxis_title="Country",
                  title_y=0.97, title_x=0.45)

fig.show()


The most important take-away from the graph above is the fact that in 2000, Japan had a higher GDP than that of China but because of China's high GDP growth rate, it overtook Japan after 2009/2010. Furthermore, another important takeaway is that the United States has had the highest GDP in the world since 2000.

**Creating a timeline graph that shows the trends of GDPs for the 181 countries in the dataset**

In [ ]:
fig = px.scatter(country_new, x="Country Name", y="Value", color="Country Name",
                 animation_frame="Year", animation_group="Country Name")

fig.update_layout(title_text="GDP Trends of Countries (2000-2016)", title_font_size=22,
                  height=800, width=2000, yaxis_title="GDP($)", xaxis_title="Country",
                  title_y=0.97, title_x=0.45)

fig.show()

This timeline graph, though has countries squeezed together on the x-axis, reveals some important information. It shows how most countries are on the baseline of GDPs that are less than $100 billion, and most importantly, it reveals the countries that have moved away from that baseline since 2000 and have improved their GDPs drastically.This includes, apart from the top 15 countries mentioned in previous graphs, countries such as Argentina, Austria, Belgium, Chile, Colombia, Indonesia, Iran, Hong Kong, Malayisa and many more.

What would help this analysis even further would be to see which countries have improved their GDPs the most during this time-period.

**Finding out the countries with the highest GDP growth rates from 2000 to 2016**

In [ ]:
countries_2000 = country_new[country_new["Year"]==2000]
countries_2000.reset_index(drop=True, inplace=True)

countries_2016 = country_new[country_new["Year"]==2016]
countries_2016.reset_index(drop=True, inplace=True)

countries_2000_value = pd.DataFrame(countries_2000["Value"])
countries_2000_value.reset_index(drop=True, inplace=True)

header=["Value 2000"]
countries_2000_value.columns = header

countries_2000_2016 = pd.concat([countries_2016, countries_2000_value], axis=1, ignore_index=False)

countries_2000_2016["GDP Growth Rate(%)"] = ((countries_2000_2016["Value"]-countries_2000_2016["Value 2000"])/countries_2000_2016["Value 2000"])

countries_2000_2016.drop(labels=["Year"], axis=1, inplace=True)

countries_2000_2016.sort_values("GDP Growth Rate(%)", axis=0, ascending=False, kind='quicksort', inplace=True)

top_GDP = countries_2000_2016.head(15)
top_GDP.reset_index(drop=True, inplace=True)

rounded = np.round(top_GDP["GDP Growth Rate(%)"], decimals=2)
rounded_GDP_growth = pd.DataFrame(rounded)
top_GDP["GDP Growth Rate(%)"] = rounded_GDP_growth


fig = px.bar(top_GDP, x="Country Name", y="GDP Growth Rate(%)", color="Country Name",
             color_discrete_sequence=px.colors.qualitative.Dark24, text="GDP Growth Rate(%)")

fig.update_layout(title_text="Top 15 Countries with the highest GDP Growth Rate % (2000-2016)", title_font_size=22,
                  height=700, width=980, yaxis_title="GDP Growth Rate(%)", xaxis_title="Country",
                  title_y=0.97, title_x=0.45)

fig2 = px.choropleth(countries_2000_2016, locations="Country Name", locationmode='country names', color="GDP Growth Rate(%)",
                    color_continuous_scale=px.colors.sequential.Darkmint)

fig2.update_layout(title_text="Heatmap of GDP Growth Rate % (2000-2016)", title_font_size=24,
                  height=800, width=1000, yaxis_title="GDP($)", xaxis_title="Country",
                  title_y=0.85, title_x=0.45)


fig.show()
fig2.show()

top_GDP

As illustrated in the table and graphs above, the highest GDP growth rates from 2000 to 2016 were experienced by countries in Central & South East Asia and some countries in Sub-Saharan Africa.

***I will be continuing this analysis further by creating a forecasting model to predict GDPs for the top 15 countries with the highest GDPs in 2016 for 2017, 2018 and 2019. Furthermore, I will be comparing these forecasts to the actual GDP values recorded for these countries, to gauge how accurate the model is.***